<a href="https://colab.research.google.com/github/mateuszrajca69/r/blob/main/projekt69.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install ortools

In [1]:
pip install --user ortools

In [3]:
#Poniższy kod importuje wymagane biblioteki.
from ortools.linear_solver import pywraplp

In [7]:
def create_data_model():
    """Create the data for the example."""
    data = {}
    weights = [48, 30, 19, 36, 36, 27, 42, 42, 36, 24, 30]    #wagi: wektor zawierający wagi elementów.
    data['weights'] = weights
    data['items'] = list(range(len(weights)))
    data['bins'] = data['items']                              
    data['bin_capacity'] = 100                                #bin_capacity: pojedyncza liczba określająca pojemność pojemników.
    return data


    #maksymalna liczba pojemników, których można potrzebować, to liczba przedmiotów, ponieważ zawsze można umieścić każdy przedmiot w oddzielnym koszu.



    
def main():
    data = create_data_model()


# zastosowano solver do programowania mieszanych liczb całkowitych. Poniższy kod deklaruje solver.
    solver = pywraplp.Solver.CreateSolver('SCIP')


   
# x [i, j] = 1, jeśli element i jest zapakowany do pojemnika j.
    x = {}
    for i in data['items']:
        for j in data['bins']:
            x[(i, j)] = solver.IntVar(0, 1, 'x_%i_%i' % (i, j))

   
# y [j] = 1, jeśli używany jest bin j.
    y = {}
    for j in data['bins']:
        y[j] = solver.IntVar(0, 1, 'y[%i]' % j)

#Podobnie jak  z wieloma plecakami, definiuje tablicę zmiennych x [(i, j)], których wartość wynosi 1, jeśli element i zostanie umieszczony w bin j, a 0 w przeciwnym razie.
#W przypadku pakowania w przedział definiuje się również tablicę zmiennych y [j], której wartość wynosi 1, jeśli używany jest przedział j - to znaczy, 
#jeśli są w niej spakowane jakiekolwiek elementy - i 0 w przeciwnym razie. Suma y [j] będzie liczbą użytych przedziałów.

 
  #Poniższy kod definiuje ograniczenia problemu:
    for i in data['items']:
        solver.Add(sum(x[i, j] for j in data['bins']) == 1)

    
# Ilość umieszczona w każdym pojemniku nie może przekraczać jego pojemności.
    for j in data['bins']:
        solver.Add(
            sum(x[(i, j)] * data['weights'][i] for i in data['items']) <= y[j] *
            data['bin_capacity'])
#Każdy przedmiot należy umieścić w dokładnie jednym pojemniku. To ograniczenie jest ustalane przez wymaganie, aby suma x [i] [j] na wszystkich pojemnikach j była równa 1. 

   
# Cel: zminimalizować liczbę używanych pojemników.
    solver.Minimize(solver.Sum([y[j] for j in data['bins']]))



#Poniższy kod wywołuje solver i wyświetla rozwiązanie.
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        num_bins = 0.
        for j in data['bins']:
            if y[j].solution_value() == 1:
                bin_items = []
                bin_weight = 0
                for i in data['items']:
                    if x[i, j].solution_value() > 0:
                        bin_items.append(i)
                        bin_weight += data['weights'][i]
                if bin_weight > 0:
                    num_bins += 1
                    print('Bin number', j)
                    print('  Items packed:', bin_items)
                    print('  Total weight:', bin_weight)
                    print()
        print()
        print('Number of bins used:', num_bins)
        print('Time = ', solver.WallTime(), ' milliseconds')
    else:
        print('The problem does not have an optimal solution.')


if __name__ == '__main__':
    main()

Bin number 0
  Items packed: [0, 1, 2]
  Total weight: 97

Bin number 1
  Items packed: [3, 4, 5]
  Total weight: 99

Bin number 2
  Items packed: [6, 7]
  Total weight: 84

Bin number 3
  Items packed: [8, 9, 10]
  Total weight: 90


Number of bins used: 4.0
Time =  20  milliseconds


In [8]:
#Rozwiązanie pokazuje minimalną liczbę pojemników potrzebną do zapakowania wszystkich artykułów. Dla każdego używanego pojemnika rozwiązanie pokazuje zapakowane w nim elementy i całkowitą wagę pojemnika.